In [32]:
# Importing Libraries for Deep Learning and Data Processing
# 
# This block imports essential libraries for building and training deep learning models using TensorFlow and Keras. 
# - `tensorflow` is the main deep learning framework, and eager execution is enabled for easier debugging and dynamic computation.
# - `numpy` and `pandas` are used for numerical operations and data manipulation.
# - `os` and `glob` help with file and directory operations.
# - `%matplotlib inline` and `matplotlib.pyplot` are for plotting and visualizations within the notebook.
# - `skimage.io` is used for image input/output operations.
# - `ImageDataGenerator` from Keras is used for real-time data augmentation and preprocessing of images.
# - Keras layers and models (`Dense`, `Dropout`, `Flatten`, `Conv2D`, `MaxPooling2D`, `Sequential`, `Model`) are used to construct neural network architectures.
# - Pre-trained models like `VGG16` and `ResNet50` are imported for transfer learning.
# - `Adam` optimizer and various callbacks (`ModelCheckpoint`, `LearningRateScheduler`, `EarlyStopping`, `ReduceLROnPlateau`) are included for model training and optimization.
#

# Enable eager execution
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# Verify eager execution is enabled
print("Eager execution:", tf.executing_eagerly())

import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

Eager execution: True


In [ ]:
# Loading Training and Validation DataFrames with Pandas
#
# This block uses the pandas library to read CSV files containing image paths and class labels for training and validation.
# - `pd.read_csv()` reads a CSV file into a DataFrame, which is a tabular data structure ideal for data manipulation and analysis.
# - `train_df` holds the training data, while `valid_df` holds the validation data.
# - Each DataFrame contains two columns: 'img_path' (the filename of the image) and 'class' (the label for each image).
# - The class labels in this dataset are 'dense' and 'fatty', representing different categories for the images.
# These DataFrames are later used to generate batches of images and labels for model training and validation.
#

train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

## Setting up the image augmentation from last Lesson: 

In [ ]:
# Setting the Image Size for Model Input
#
# This block defines the target image size for all images fed into the neural network.
# The variable `IMG_SIZE` is set as a tuple (224, 224), which is a common input size for models like VGG16.
# This ensures that all images are resized to 224x224 pixels before being processed by the model.

IMG_SIZE = (224, 224)

In [ ]:
# Image Augmentation and Data Generator Setup with Keras
#
# This block uses Keras' ImageDataGenerator to preprocess and augment image data for deep learning.
# - ImageDataGenerator: Provides real-time data augmentation (random transformations) and normalization for training images.
#   - For training data, we apply rescaling, random horizontal flips, shifts, rotations, shearing, and zooming to increase dataset diversity.
#   - For validation data, only rescaling is applied to ensure evaluation on unaltered images.
# - flow_from_dataframe: Loads images and labels from a pandas DataFrame, generating batches for model training/validation.
#   - x_col and y_col specify the columns for image paths and labels.
#   - class_mode='binary' is used for binary classification tasks.
#   - target_size resizes images to the required input shape for the model.
#   - batch_size controls the number of images per batch.

train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )


# Validation Data Generator Setup with Keras ImageDataGenerator
#
# This section uses Keras' ImageDataGenerator to preprocess validation images for model evaluation.
# - `val_idg` is an instance of ImageDataGenerator configured only with rescaling (normalizing pixel values to [0, 1]).
#   - No augmentation is applied to validation data to ensure that model performance is evaluated on unaltered images.
# - `val_gen` is a DataFrameIterator created by calling `flow_from_dataframe` on `val_idg`.
#   - It loads images and labels from the `valid_df` DataFrame.
#   - `x_col` and `y_col` specify the columns for image file paths and class labels.
#   - `class_mode='binary'` is used for binary classification.
#   - `target_size` resizes images to the required input shape for the model.
#   - `batch_size` sets the number of images per batch (6 in this case, matching the number of validation images).
#
# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [ ]:
# Pulling a Batch of Validation Data for Model Evaluation
#
# This block demonstrates how to use Keras' DataFrameIterator (created by ImageDataGenerator's `flow_from_dataframe`)
# to retrieve a batch of validation images and labels. The `next()` function is called on `val_gen`, which yields a tuple:
# - `testX`: a batch of preprocessed image data (as a NumPy array)
# - `testY`: the corresponding labels for the images
# This batch can be used for evaluating model predictions or visualizing results after each training epoch.

## Pull a single large batch of random validation data for testing after each epoch
testX, testY = next(val_gen)

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [33]:
# Loading the VGG16 Pre-trained Model with Keras
#
# This block demonstrates how to load the VGG16 model using Keras, a high-level neural networks API.
# - `VGG16` is a popular convolutional neural network architecture that has been pre-trained on the ImageNet dataset.
# - `include_top=True` loads the fully connected layers at the top of the network, which are used for classification.
# - `weights='imagenet'` loads the weights trained on the ImageNet dataset, allowing for transfer learning or feature extraction.
# - `model.summary()` prints a summary of the model architecture, including each layer and the number of parameters.
#

model = VGG16(include_top=True, weights='imagenet')
model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Extracting Intermediate Layer Output from VGG16 using Keras Functional API
#
# This block demonstrates how to use the Keras Functional API to create a new model that outputs the activations from an intermediate layer of a pre-trained VGG16 model.
# - `model.get_layer('block5_pool')` retrieves the 'block5_pool' layer from the loaded VGG16 model. This is the last max pooling layer before the fully connected layers.
# - `Model(inputs=model.input, outputs=transfer_layer.output)` creates a new Keras model (`vgg_model`) that takes the same input as the original VGG16 model but outputs the feature maps from the 'block5_pool' layer.
# This approach is commonly used for transfer learning, where the convolutional base is used as a feature extractor for new tasks.

#

transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [35]:
# Fine-Tuning VGG16 Layers with Keras
#
# This block demonstrates how to selectively fine-tune layers of a pre-trained VGG16 model using Keras.
# - Keras allows you to set the `trainable` attribute of each layer to control whether its weights are updated during training.
# - By setting `layer.trainable = False`, the weights of that layer are frozen and will not be updated.
# - In this example, all layers except the last convolutional block (layers 17 and above) are frozen, allowing only the deeper layers to be fine-tuned on the new dataset.
# - This approach leverages pre-learned features from ImageNet while adapting the model to the specifics of your data.
#

## Now, choose which layers of VGG16 we actually want to fine-tune (if any)
## Here, we'll freeze all but the last convolutional layer
for layer in vgg_model.layers[0:17]:
    layer.trainable = False

In [36]:
# Inspecting Trainable Layers in a Keras Model
#
# This block demonstrates how to inspect which layers of a Keras model are set as trainable.
# - Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow.
# - Each layer in a Keras model has a `trainable` attribute that determines whether its weights will be updated during training.
# - Iterating through `vgg_model.layers` allows us to print the name of each layer along with its trainable status.
# This is useful for verifying which layers are frozen (not trainable) and which are being fine-tuned, especially when using transfer learning.
#

for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_layer_5 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


In [37]:
# Building a Simple Transfer Learning Model with Keras Sequential API
#
# This block demonstrates how to build a binary image classification model using transfer learning with Keras.
# - The `Sequential` model from Keras is used to stack layers linearly.
# - The pre-trained convolutional base (`vgg_model`), which outputs feature maps from VGG16 up to 'block5_pool', is added as the first layer.
# - `Flatten()` converts the 2D feature maps into a 1D vector for the dense layer.
# - A single `Dense` layer with a sigmoid activation is added for binary classification, outputting a probability.
# This approach leverages pre-trained features from VGG16 and adds a simple classifier on top for the new task.

#

new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='sigmoid'))

In [38]:
# Setting Up Optimizer, Loss Function, and Metrics for Model Training
#
# This block configures the key components required to compile a Keras deep learning model:
# - The Adam optimizer (`Adam`) is initialized with a learning rate of 1e-4. Adam is an adaptive learning rate optimization algorithm widely used for training deep neural networks.
# - The loss function is set to 'binary_crossentropy', which is appropriate for binary classification tasks.
# - The metrics list includes 'binary_accuracy', which tracks the proportion of correctly classified samples during training and evaluation.
# These settings are essential for guiding the model's learning process and evaluating its performance.

#

## Set our optimizer, loss function, and learning rate
optimizer = Adam(learning_rate=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [39]:
# Compiling a Keras Model for Training
#
# This block uses the Keras deep learning library to prepare a neural network model for training.
# - `compile()` is a method in Keras that configures the model for training by specifying:
#   - The optimizer (e.g., Adam, SGD) which determines how the model weights are updated based on the loss function.
#   - The loss function (e.g., 'binary_crossentropy') which measures how well the model's predictions match the true labels.
#   - The metrics (e.g., 'binary_accuracy') which are used to evaluate the model's performance during training and testing.
# Compiling is a required step before fitting the model to data with `fit()`.

#

new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Training the Transfer Learning Model with Keras
#
# This block demonstrates how to train a deep learning model using the Keras library, which is a high-level API for building and training neural networks in Python.
# - `new_model.fit()` is the main function for training a Keras model. It takes in the training data, validation data, and the number of epochs to train for.
#   - `train_gen` is a data generator that yields batches of augmented and preprocessed training images and labels.
#   - `validation_data=(testX, testY)` provides a batch of validation images and labels to evaluate the model's performance after each epoch.
#   - `epochs=5` specifies that the model will be trained for 5 complete passes through the training data.
# This approach leverages Keras' easy-to-use interface for model training and evaluation, making it simple to experiment with different architectures and training strategies.
#


## Just run a single epoch to see how it does:
new_model.fit(train_gen,
              validation_data=(testX, testY),
              epochs=5)

/Users/anup/anup_prompt_venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 5s/step - binary_accuracy: 0.4886 - loss: 0.7368 - val_binary_accuracy: 0.6667 - val_loss: 0.6367
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 8s/step - binary_accuracy: 0.6740 - loss: 0.6687 - val_binary_accuracy: 0.5000 - val_loss: 0.6430
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 5s/step - binary_accuracy: 0.6030 - loss: 0.6346 - val_binary_accuracy: 0.8333 - val_loss: 0.5895
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 9s/step - binary_accuracy: 0.8295 - loss: 0.5262 - val_binary_accuracy: 0.6667 - val_loss: 0.6380
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 6s/step - binary_accuracy: 0.6583 - loss: 0.6320 - val_binary_accuracy: 0.6667 - val_loss: 0.6941


## Let's try another experiment where we add a few more dense layers:

In [29]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [30]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [31]:
## Just run a single epoch to see how it does:
# Replace fit_generator() with fit()
new_model.fit(train_gen,
              validation_data=(testX, testY),
              epochs=5)

Epoch 1/5


ValueError: Unknown variable: <Variable path=sequential_3/dense_7/kernel, shape=(25088, 1024), dtype=float32, value=[[-0.00764912  0.00930794 -0.00369001 ... -0.0012937   0.0112356
  -0.01313087]
 [-0.01469244 -0.00710067  0.01310301 ... -0.00278975  0.00539955
  -0.01398823]
 [-0.0111308   0.01017589  0.00072407 ... -0.000337    0.00967058
  -0.00360134]
 ...
 [-0.01478224 -0.01100576  0.01095728 ...  0.01346205  0.00826249
  -0.00815834]
 [ 0.01461284  0.00071985 -0.00305918 ...  0.00064543 -0.00825646
   0.00270579]
 [-0.01384913  0.01099334 -0.00124807 ... -0.01334289 -0.00075179
  -0.01164355]]>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

## Now let's add dropout and another fully connected layer:

In [ ]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(256, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

What's interesting about the small number of epochs we ran on the three different architectures above is that the simplest archiecture seemed to show the fastest learning. Why might that be? 

Answer: there were the fewest parameters to train because we didn't add any fully-connected layers, and were only fine-tuning the last layer of VGG16. 

The last architecture we tried seemed to show more stable and promise than the second, and this is likely due to the fact that we added Dropout. This helps our model from overfitting and usually using Dropout, we see better learning on the validation set (val_loss going down over epochs as opposed to only the training loss). 

# Summary of the Script

This notebook demonstrates a complete workflow for binary image classification using transfer learning with the VGG16 architecture in Keras. The process begins with importing essential libraries for deep learning, data processing, and visualization. Training and validation datasets are loaded from CSV files into pandas DataFrames, which are then used to generate batches of images with real-time augmentation for training and simple rescaling for validation. The VGG16 model, pre-trained on ImageNet, is loaded and modified to output features from the 'block5_pool' layer, enabling transfer learning. Most layers are frozen to retain learned features, while only the last convolutional block is fine-tuned. Several model architectures are explored by stacking additional dense and dropout layers to improve generalization and prevent overfitting. The models are compiled with the Adam optimizer and binary cross-entropy loss, and trained using the prepared data generators. The notebook highlights the trade-offs between model complexity and learning speed, showing that simpler architectures may converge faster with fewer parameters, while dropout layers help stabilize training and reduce overfitting. This workflow provides a practical foundation for leveraging pre-trained models on custom image datasets, emphasizing best practices in data augmentation, transfer learning, and model evaluation.